### GASS evolution

nb to test the evolution functions 

In [9]:
using PyCall
using DataFrames
using Printf

#rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

@pyimport astropy.coordinates as coord
@pyimport astropy.modeling.models as am
@pyimport astropy.modeling.fitting as mf

@pyimport numpy as np


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [10]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    # prop::Array{Array{Float64,2},2}     ## beam and mrs for each subarray (to be confirmed)    
end

In [11]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res)
end

_fitness_subarray (generic function with 1 method)

In [12]:
## Creation pf a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j]= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(0, pop, fitness, score)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [13]:
## get best parents
##

function _get_elitism(cfg, pop::_population)
    isort= reverse(sortperm(pop.score))
    
    elit= []
    for i in 1:cfg.ga.Number_Elitism
        push!(elit, [pop.subarr[isort[i],:], pop.fitness[isort[i],:], pop.score[isort[i]]])
    end
    return(elit)
end

## using tournament selection
function _get_parents(cfg, pop::_population)
    tour= cfg.ga.Tournament_Size
    
    popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
    isort= reverse(sortperm(pop.score[popran[1:tour]]))
    iparent1= popran[isort[1]]
  
    iparent2= iparent1
    while iparent1==iparent2
        popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
        isort= reverse(sortperm(pop.score[popran[1:tour]]))
        iparent2= popran[isort[1]] 
    end 
    return(iparent1 , iparent2)
end

_get_parents (generic function with 1 method)

In [14]:
function _flatten_subarray(cfg, subarr)
    subflat= zeros(Int, cfg.obs.Antenna_Number)
    
    counter= 1
    for i in 1:cfg.obs.Subarray_Number
        for j in 1:cfg.sub.Pads_Per_Subarray[i]
            subflat[counter]= subarr[i][j]
            counter += 1
        end
    end
    return(subflat)
end

## reform the subarray array..
function _wrap_subarray(cfg, sub)
    subarray= Array{Array{Int,1}}(undef,0)

    for i in 1:cfg.obs.Subarray_Number
        push!(subarray, sub[cfg.sub.Subrange[i]])
    end
    
    return(subarray)
end

## crossover using the two parents
##
function _get_crossover1(cfg, pop::_population , parents)
    nmax= cfg.obs.Antenna_Number
    p1= _flatten_subarray(cfg, pop.subarr[parents[1],:])
    p2= _flatten_subarray(cfg, pop.subarr[parents[2],:])
    child= zeros(Int, nmax)

    pivot= rand(1:nmax,2)
    pivot1= minimum(pivot) ; pivot2= maximum(pivot)
    child[pivot1:pivot2] .= p1[pivot1:pivot2]

    irange1= 1:pivot1-1 ; irange2= pivot2+1:nmax
    
    cnterp2= 1
    for i in irange1
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end
    for i in irange2
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end  
    
    println(child)
    c= _wrap_subarray(cfg, child)
    println(c)
    return(child)
end

_get_crossover1 (generic function with 1 method)

In [15]:
## mutation
## loop over all alleles and swap two if p is true
## 

function _get_mutation(cfg,pop::_population , indiv)
    nmax= cfg.obs.Antenna_Number
    si= _flatten_subarray(cfg, pop.subarr[indiv,:])
    
    pmutation= 1-cfg.ga.Mutation_Rate/100
    rng = MersenneTwister(1234)
    rd= rand(rng, nmax)
    
    for i in 1:nmax
        if rd[i] > pmutation
            print("mute")
        end
    end
end

_get_mutation (generic function with 1 method)

In [16]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Creating the population...")
    p0= _create_population(cfg)
    pelit=  _get_elitism(cfg, p0)
    parent= _get_parents(cfg, p0)
    child=  _get_crossover1(cfg, p0 , parent)
    
    
    #println(pelit)
    
    ## PLOT ####
    #PyPlot.imshow(h)
    #PyPlot.colorbar()
    #PyPlot.show()
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 100 
### Population size: 1000 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Creating the population...
1 1 -3.2234773564726122
1 2 1.0559706820574324
1 3 -11.460839139010611
1 4 10.314644173724075
2 1 2.506050905869942
2 2 0.889855193671524

76 4 10.444843883960974
77 1 0.7327876300387073
77 2 -0.44920626230306704
77 3 -13.858658352269359
77 4 12.753273076421177
78 1 0.1559218334787753
78 2 -1.4070797114657863
78 3 -13.785087520958829
78 4 11.099914114552103
79 1 1.8510088048990359
79 2 0.13543731686595217
79 3 -17.09868281319202
79 4 -10.881777270170508
80 1 1.6342398083097844
80 2 0.585655599770609
80 3 3.513730956631448
80 4 9.540911809837372
81 1 1.6992108994742328
81 2 1.337944298395592
81 3 -11.16033194226548
81 4 10.936992430982711
82 1 2.037802099173909
82 2 1.1984559272810391
82 3 -11.348224456740535
82 4 1662.491208436488
83 1 2.3482514245348263
83 2 0.8297602724674826
83 3 -12.428823636224696
83 4 9.33731915476677
84 1 2.543271563689954
84 2 -0.05571206369742887
84 3 -11.961862154130204
84 4 5.757267842490904
85 1 1.1601239531203338
85 2 -2.0162748449504413
85 3 -13.368401508496312
85 4 15.8206167819924
86 1 2.5366802334805616
86 2 -0.552635624951777
86 3 -12.218385223198617
86 4 7.351324551015868
87 1 1.4304188

159 1 1.8173945551189383
159 2 1.8518726877511966
159 3 -12.826820054950966
159 4 12.009000213245757
160 1 2.0385128755459663
160 2 0.7843732355034749
160 3 -13.614416166225526
160 4 -10.212494341809958
161 1 2.465907229697683
161 2 -1.2496635503444673
161 3 -12.316358091797
161 4 13.001492661230447
162 1 2.2590490270232806
162 2 -3.157873959419339
162 3 -10.000865017629447
162 4 12.371706992533152
163 1 2.238316155415879
163 2 1.1097086320069223
163 3 -11.90197946233566
163 4 2.7094944847113136
164 1 0.2560909781900699
164 2 1.246442778569754
164 3 -11.63339275627284
164 4 22.818344987528647
165 1 1.8018378738260752
165 2 0.7303690969189702
165 3 -10.792713204001148
165 4 -0.5375338404095786
166 1 1.5763089726867208
166 2 1.180920434943801
166 3 -10.752072242608309
166 4 11.63102549535778
167 1 -6.6322401255189884
167 2 1.008017194341098
167 3 -6.363003531998807
167 4 -13.047648537606548
168 1 -0.5643884934270421
168 2 0.16614583013619333
168 3 -9.774934914061847
168 4 5.9387238787614

240 3 -11.226644317572358
240 4 3.871791578287324
241 1 -1.3080322558673725
241 2 -2.1658538273990153
241 3 -10.279171195038963
241 4 -11.264518746402688
242 1 -1.460123191601919
242 2 -9.109018558583616
242 3 -14.187223892696474
242 4 137268.43181602858
243 1 1.4963140315953558
243 2 0.9806333588873253
243 3 -13.208644061394862
243 4 11.04072916663829
244 1 -2.2369428627747365
244 2 -3.396024737638052
244 3 -16.421996332571197
244 4 16.71023265862929
245 1 0.7315510347513616
245 2 -2.3521838629126153
245 3 -11.152408221914214
245 4 -14.39040135821218
246 1 2.3178034603765374
246 2 1.819964498568074
246 3 -14.718433428231815
246 4 10.204175702082523
247 1 2.1833205739070696
247 2 -82.7184027202015
247 3 -12.009232157358202
247 4 -8.070437421791764
248 1 -2.368778723579295
248 2 0.94559556022294
248 3 -4.738715749876018
248 4 -16.91599880434406
249 1 1.4734977769366266
249 2 -1.3229593261350314
249 3 -12.581301207033412
249 4 12.041928312324798
250 1 0.7332201194364741
250 2 1.451194780

322 1 2.2996943082008183
322 2 1.5419508958556807
322 3 -11.226618774990229
322 4 -15.582431070426043
323 1 -0.24024250334551422
323 2 0.15722085220567683
323 3 -14.733805938708457
323 4 5.025912492537177
324 1 -2.052514265117255
324 2 0.8589801993979966
324 3 -15.371372111223671
324 4 -12.193239321100048
325 1 -16.084578286049123
325 2 -0.4736528066557768
325 3 -12.83943570206568
325 4 -20.660223056129205
326 1 0.14571674152541614
326 2 -0.7568785960555204
326 3 -6.690911573983172
326 4 92272.26710352457
327 1 -5.688121208791804
327 2 0.9596346267028295
327 3 -13.64084374612969
327 4 24.984482856302538
328 1 0.8269210939101226
328 2 -2.929649042318035
328 3 -31.757468430650885
328 4 12.49075083352363
329 1 -0.7389642523288835
329 2 0.8305147166462334
329 3 -17.85771121775238
329 4 6.7081672620661985
330 1 2.3581295109802305
330 2 -1.861747911786869
330 3 -12.582593775167258
330 4 30.42556318343954
331 1 2.7817072719351152
331 2 -4.884389990102893
331 3 -11.382767945011487
331 4 19.393

...

728 3 -12.757495455330057
728 4 -13.759953234605343
729 1 1.4749192830322007
729 2 -0.2986339382405574
729 3 -13.016886299529952
729 4 63410.46567338043
730 1 2.218065555801688
730 2 0.7821943971249685
730 3 -13.740128845321863
730 4 6.105356653682845
731 1 1.5919815286493975
731 2 0.7219831444482958
731 3 -13.5480311569125
731 4 -9.724047041540478
732 1 2.0003576504387923
732 2 -0.6064553190149526
732 3 -9.943714688634333
732 4 6.272082218383208
733 1 2.6547874465542387
733 2 -1.0099489912149249
733 3 -10.880978480607434
733 4 9.892014694922526
734 1 3.137480627566857
734 2 -0.25014685464176845
734 3 -12.92509555947415
734 4 11.269131450306629
735 1 -4.004043155057762
735 2 0.4293100159210659
735 3 -13.408257095342991
735 4 11.761859480260753
736 1 -4.5427517451206185
736 2 -2.1196636673183025
736 3 -10.285209727705604
736 4 -2.9268661522145347
737 1 2.006988765620427
737 2 0.45827216207678445
737 3 -13.581140128871985
737 4 4.5762569677285745
738 1 2.4564593741983876
738 2 1.07784202

809 4 9.938915665225611
810 1 3.2582192375656276
810 2 -23.12336983073778
810 3 -11.01598158029713
810 4 7.876568563397171
811 1 1.7921292638390198
811 2 -0.6102808937971411
811 3 -12.109719316561037
811 4 66240.33873901564
812 1 2.3743581022368874
812 2 -0.4540814171422629
812 3 -23.383989637636972
812 4 12.637450968907208
813 1 -0.8675642644620309
813 2 -0.7957091951771527
813 3 -11.19418196811689
813 4 -9.459813042669229
814 1 2.557322469753987
814 2 0.5144944499988315
814 3 -13.41208407073994
814 4 13.10921095479263
815 1 2.7015902066348834
815 2 1.3093653945921324
815 3 -19.888796970301073
815 4 11.893011560889516
816 1 0.7377981648430734
816 2 0.27054261975175153
816 3 -6.8438809402479235
816 4 10.75046371646734
817 1 1.8724590742488845
817 2 2.6168182206516004
817 3 -12.134303180980686
817 4 43431.34165900327
818 1 -4.5357298413065585
818 2 -1.0693294216512728
818 3 -12.65487496819819
818 4 12.592395442084563
819 1 3.1396477675220633
819 2 1.110772048785535
819 3 -12.03178589133

891 1 2.9167235287093494
891 2 -0.24038047258907547
891 3 -10.485319164311242
891 4 17.708476204139856
892 1 1.699583705881458
892 2 -0.863037271577197
892 3 -10.358305177034723
892 4 -10.531922697980107
893 1 2.1230257056299546
893 2 -1.2199826847335595
893 3 -7.57371691858963
893 4 14.301197452706798
894 1 1.3464780190644383
894 2 -2.3171752612614784
894 3 -12.926606868035663
894 4 -11.876619551604884
895 1 -3.536804386268482
895 2 -23.84037343832148
895 3 -11.275099007995156
895 4 2.9224712074834756
896 1 1.7622948459232113
896 2 0.6481796559160915
896 3 -11.386111334682777
896 4 5.470397588106035
897 1 2.227280155382631
897 2 0.3037123954020884
897 3 -11.22633066431671
897 4 -9.093879370930914
898 1 3.055370465786253
898 2 -3.3591619009859186
898 3 -11.567144279438736
898 4 18.237542612244713
899 1 -0.49966966646989974
899 2 -3.4525105825861733
899 3 -11.705399034376981
899 4 -12.342846919142508
900 1 0.12174963254881632
900 2 0.17413866958583757
900 3 -20.940881617590886
900 4 -11

972 2 0.7210216740596027
972 3 -12.929726650474162
972 4 43430.853881885436
973 1 0.4916346872673112
973 2 1.0587733566033033
973 3 -14.622627997717466
973 4 10.496825539869146
974 1 0.8874536561004335
974 2 -19.90322403486802
974 3 -8.622678055263808
974 4 20.279365876112173
975 1 1.2431140883955158
975 2 1.1478151417007663
975 3 -13.604513107709586
975 4 2.5754225000986604
976 1 -2.868041343327336
976 2 0.36969716793259444
976 3 -13.749746396919628
976 4 12.51078455490753
977 1 2.33593339123224
977 2 0.8766908848350621
977 3 -11.935020783129996
977 4 15059.305300639038
978 1 2.810890731922857
978 2 -4.155237235366742
978 3 -9.739784983026171
978 4 -13.086732706440612
979 1 1.7646547554280945
979 2 0.9099352661095237
979 3 -12.411735715654212
979 4 -10.108247498887367
980 1 -6.589477769946659
980 2 -2.20268636253318
980 3 -9.626748308966198
980 4 -12.454679703594753
981 1 2.365075835947011
981 2 -1.149443773882553
981 3 -12.344155435857466
981 4 7.924086396595522
982 1 1.9980486994969

50-element Array{Int64,1}:
 33
 32
 30
 14
 23
 46
 42
  3
 40
 35
 16
 24
 26
  ⋮
 50
 34
 19
 49
 15
 28
  9
  4
 13
  6
 31
 38